In [17]:
import numpy as np
import pandas as pd
import copy
import pickle
import random

In [18]:
def preprocess_text(text):
    '''получаем из текста список слов'''
    text = text.str.lower()
    #в регулярном выражении \d означает "любая цифра" \W - не цифра и не буква
    #и + означает "один или более text = text.str.replace('\d+|\W+',' ')
    text = text.str.replace('_',' ')
    #text = text.str.replace('@',' абазин ')
    #text = text.str.replace('\B,','_')
    text = text.str.replace('\d+|\W+',' ')
    text = text.str.replace('_',' ')
    text = text.str.replace('ё','е')
    text = text.str.strip()
    text = text.str.split()
    return text

In [19]:
# загрузка файла
path_to_folder = r"C:\Users\Dima\Desktop\СберУниверситет\Дипломный проект\Диплом файлы"
file_to_load = "HeadHunter_train_dummy.csv"
path = path_to_folder + "//" + file_to_load

file_df = pd.read_csv(path)

In [20]:
file_df

,review_id,city,position,positive,negative,salary_rating,team_rating,managment_rating,career_rating,workplace_rating,rest_recovery_rating,target
0,0,Ижевск,"Мастер контрольный (участка, цеха)","мила дружка семь верст не околица, - заговорил...","более ради оскорбления, чем в точном смысле......",5,4,3,3,4,4,0
1,1,Москва,Инженер лаборатории,"прибавь, как раз так будет, - закричал он тоне...",- промелькнуло в нем как молния.\n - Извин...,4,4,4,1,1,1,8
2,2,Екатеринбург,Техник по документации плавучей атомной станции,"и пропустил ее вперед. Седой Тихон, в парике, ...","- Да, письмо; а ты не знал? Гм.\n",5,5,5,5,5,5,0
3,5,Санкт-Петербург,Техник-полиграфист,"- Нет, его так не уломаешь ни за что, - гов...","Стоило ль, например, стараться интриговать, чт...",5,5,5,5,5,5,0
4,7,Москва,Инструктор-переводчик по обслуживанию глухих р...,"нахмурившись не столько на капитана, сколько н...","модной ходячей идее, чтобы тотчас же опошлить ...",4,5,4,2,5,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...
50871,192815,Челябинск,Инженер по эксплуатации и наладке оборудования...,нами повелителем. Не говоря уже о моих братьях...,извините меня за мою назойливость (беспокою уж...,2,2,1,1,1,1,"3,8"
50872,192818,Санкт-Петербург,Инженер-физик в ракетно-космической промышленн...,задумался и поморщился. Анна Михайловна поняла...,"нему посмотреть. Только ничего, вздор, всё хор...",5,3,4,3,5,2,8
50873,192820,Нижний Новгород,Лаборант-исследователь (в области физики),"- Я вам прямо скажу, - сказал князь Васили...","всё это чрезвычайно интересует, и почту за дол...",2,1,3,2,2,1,8
50874,192827,Москва,Инженер-проектировщик фотошаблонов для произво...,"так, я не знаю. Неловко, стыдно.\n - А я з...",\n195\n\n милый друг (франц.). \n \n...,4,5,1,1,1,1,1


In [21]:
# при обработке тестового набора убрать
#file_df.drop_duplicates(subset=['positive', 'negative',
#       'salary_rating', 'team_rating', 'managment_rating', 'career_rating',
#       'workplace_rating', 'rest_recovery_rating'], inplace=True, ignore_index=True)
file_df.drop_duplicates(subset=['positive', 'negative'], inplace=True, ignore_index=True)
file_df

,review_id,city,position,positive,negative,salary_rating,team_rating,managment_rating,career_rating,workplace_rating,rest_recovery_rating,target
0,0,Ижевск,"Мастер контрольный (участка, цеха)","мила дружка семь верст не околица, - заговорил...","более ради оскорбления, чем в точном смысле......",5,4,3,3,4,4,0
1,1,Москва,Инженер лаборатории,"прибавь, как раз так будет, - закричал он тоне...",- промелькнуло в нем как молния.\n - Извин...,4,4,4,1,1,1,8
2,2,Екатеринбург,Техник по документации плавучей атомной станции,"и пропустил ее вперед. Седой Тихон, в парике, ...","- Да, письмо; а ты не знал? Гм.\n",5,5,5,5,5,5,0
3,5,Санкт-Петербург,Техник-полиграфист,"- Нет, его так не уломаешь ни за что, - гов...","Стоило ль, например, стараться интриговать, чт...",5,5,5,5,5,5,0
4,7,Москва,Инструктор-переводчик по обслуживанию глухих р...,"нахмурившись не столько на капитана, сколько н...","модной ходячей идее, чтобы тотчас же опошлить ...",4,5,4,2,5,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...
50871,192815,Челябинск,Инженер по эксплуатации и наладке оборудования...,нами повелителем. Не говоря уже о моих братьях...,извините меня за мою назойливость (беспокою уж...,2,2,1,1,1,1,"3,8"
50872,192818,Санкт-Петербург,Инженер-физик в ракетно-космической промышленн...,задумался и поморщился. Анна Михайловна поняла...,"нему посмотреть. Только ничего, вздор, всё хор...",5,3,4,3,5,2,8
50873,192820,Нижний Новгород,Лаборант-исследователь (в области физики),"- Я вам прямо скажу, - сказал князь Васили...","всё это чрезвычайно интересует, и почту за дол...",2,1,3,2,2,1,8
50874,192827,Москва,Инженер-проектировщик фотошаблонов для произво...,"так, я не знаю. Неловко, стыдно.\n - А я з...",\n195\n\n милый друг (франц.). \n \n...,4,5,1,1,1,1,1


In [22]:
# заменяем NA на названия столбцов
for key in file_df.keys()[1:5]:
    file_df[key].fillna(value=key, inplace=True)    

In [23]:
# препроцессинг: токенизация
file_df['city'] = preprocess_text(file_df['city'])
file_df['position'] = preprocess_text(file_df['position'])
file_df['positive'] = preprocess_text(file_df['positive'])
file_df['negative'] = preprocess_text(file_df['negative'])

<ipython-input-18-f17498e24f0c>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace('\d+|\W+',' ')


In [24]:
# заполняем пустые списки, появившиеся после обработки
for l in file_df['negative']:
    if len(l) == 0:
        l.append('negative')
for l in file_df['positive']:
    if len(l) == 0:
        l.append('positive')
s = 0
for l in file_df['negative']:
    if len(l) == 0:
        s += 1
for l in file_df['positive']:
    if len(l) == 0:
        s += 1
s

0

In [25]:
# загружаем словарь для перевода слов русского языка в основную форму
path_to_dict_rus = r'C:\Users\Dima\Desktop\СберУниверситет\Дипломный проект\Диплом файлы\words_relation_for_lema.csv'
dict_rus = pd.read_csv(path_to_dict_rus)
dict_rus['word'] = dict_rus['word'].str.replace('ё','е')
dict_rus['word_stem'] = dict_rus['word_stem'].str.replace('ё','е')
#dict_rus.sort_values(by=['word'])
dict_rus.set_index('word', inplace=True)
dict_rus.sort_index(inplace=True)

In [26]:
dict_rus

,word_stem
word,
а,а
аббат,аббат
аббата,аббат
аббатом,аббат
авангард,авангард
...,...
ящика,ящик
ящиками,ящик
ящике,ящик


In [27]:
# переводим слова в основные формы, добавляем два новых столбца в file_df
# path_to_load_words = r'C:\Users\Dima\Desktop\СберУниверситет\Дипломный проект\en_ru_dic_lmmatized.csv'
# en_rus_dic = pd.read_csv(path_to_load_words, index_col=0)

stop_words = ['без', 'более', 'бы', 'был', 'была', 'были', 'было', 'быть', 'в', 'вам', 'вас', 
              'ведь', 'весь', 'вдоль', 'вместо', 'вне', 'вниз', 'внизу', 'внутри', 'во', 
              'вокруг', 'вот', 'все', 'всегда', 'всего', 'всех', 'вы', 'где', 'да', 'давай', 
              'давать', 'даже', 'для', 'до', 'достаточно', 'его', 'ее', 'ее', 'если', 'есть', 
              'еще', 'же', 'за', 'здесь', 'из', 'и', 'или', 'им', 'иметь', 
              'их', 'как', 'то', 'кто', 'когда', 'кроме', 'кто', 'ли', 'либо', 'мне', 'может', 
              'мои', 'мой', 'мы', 'на', 'навсегда', 'над', 'надо', 'наш', 'не', 'него', 'нее', 'нет', 
              'ни', 'них', 'но', 'ну', 'об', 'однако', 'он', 'она', 'они', 'оно', 'от', 'отчего', 'очень', 
              'по', 'под', 'после', 'потому', 'что', 'почти', 'при', 'про', 'снова', 'со', 'так', 
              'также', 'такие', 'такой', 'там', 'те', 'тем', 'то', 'того', 'тоже', 'той', 'только', 'том', 
              'тут', 'ты', 'уже', 'хотя', 'чего', 'чей', 'чем', 'что', 'чтобы', 'чье', 'чья', 'эта', 'эти', 'это', 
              "а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто",
              "буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная",
              "важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали",
              "вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видеть","вместе",
              "вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать",
              "восемь","восьмой","вот","впрочем","времени","время","все","всегда","всего","всем","всеми",
              "всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный",
              "глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да",
              "давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый",
              "двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать",
              "девятый","девять","действительно","дел","делать","дело","день","деньги","десятый","десять",
              "для","до","довольно","долго","должно","должный","дом","дорога","друг","другая","другие",
              "других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если",
              "есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за",
              "занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать",
              "значит","значить","и","идти","из","или","им","именно","иметь","ими","имя","иногда",
              "их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой",
              "кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого",
              "которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли",
              "лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля",
              "менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много",
              "многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог",
              "могут","мож","может","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы",
              "на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ",
              "нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко",
              "нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко",
              "несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда",
              "никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь",
              "ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать",
              "однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно",
              "опять","особенно","остаться","от","ответить","отец","отовсюду","отсюда","очень","первый",
              "перед","писать","плечо","по","под","подумать","пожалуйста","позже","пойти","пока",
              "пол","получить","помнить","понимать","понять","пор","пора","после","последний",
              "посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при",
              "про","просто","против","процентов","пятнадцатый","пятнадцать","пятый","пять",
              "работа","работать","раз","разве","рано","раньше","ребенок","решить","россия",
              "рука","русский","ряд","рядом","с","сам","сама","сами","самим","самими","самих",
              "само","самого","самой","самом","самому","саму","самый","свет","свое","своего",
              "своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня",
              "седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал",
              "сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова",
              "со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый",
              "стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая",
              "также","таки","такие","такое","такой","там","твой","твоя","твоё","те","тебе","тебя",
              "тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только",
              "том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты",
              "тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотеть","хоть",
              "хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через",
              "четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть",
              "шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих",
              "это","этого","этой","этом","этому","этот","эту","я"] 

list_negative = copy.deepcopy(list(file_df['negative']))
list_positive = copy.deepcopy(list(file_df['positive']))

for lis in list_negative:
    for i, word in enumerate(lis):
        if word in dict_rus.index:
            if word not in stop_words:
                lis[i] = str(dict_rus.loc[word].values[0]) + 'neg'
        else:
            lis[i] = '' #lis[i] = 'йцук' + 'neg'
    lis = list(set(lis)) # удаляем повторения слов в тексте

for lis in list_positive:
    for i, word in enumerate(lis):
        if (word in dict_rus.index and word not in stop_words):
            if word not in stop_words:
                lis[i] = str(dict_rus.loc[word].values[0]) + 'pos'
        else:
            lis[i] = '' #lis[i] = 'йцук' + 'pos'
    lis = list(set(lis)) # удаляем повторения слов в тексте

file_df['positive_1'] = list_positive
file_df['negative_1'] = list_negative

In [28]:
lis

['',
 'видетьpos',
 'лепешкаpos',
 'выражениеpos',
 'отбритьpos',
 'расшибитьpos',
 'важноpos',
 'понравитьсяpos',
 'офицерpos']

In [29]:
# добавляем к числовым признакам названия столбцов
k = list(file_df.keys()[5:11])
for ind in k:
    i = ind.split('_')[0]
    file_df[ind] = file_df[ind].apply(lambda x: (i + str(x)).split())
labels = []

for x in file_df['target']:
    labels.append(len(x))
file_df['target_len'] = labels

In [30]:
file_df.keys()

Index(['review_id', 'city', 'position', 'positive', 'negative',
       'salary_rating', 'team_rating', 'managment_rating', 'career_rating',
       'workplace_rating', 'rest_recovery_rating', 'target', 'positive_1',
       'negative_1', 'target_len'],
      dtype='object')

In [31]:
lis = []
for i, key_1 in enumerate(file_df.keys()[5:11]):    
    for key_2 in file_df.keys()[(5 + i + 1):11]:
        lis1 = [[] for xc in range(len(file_df))]
        for j in range(len(file_df)):
            w1 = file_df[key_1].iloc[j]
            w2 = file_df[key_2].iloc[j]
            lis1[j].append(w1[0] + w2[0])
        lis.append(lis1)
for jj in range(len(lis)):
    file_df[str(jj)] = lis[jj]

In [32]:
file_df

,review_id,city,position,positive,negative,salary_rating,team_rating,managment_rating,career_rating,workplace_rating,...,5,6,7,8,9,10,11,12,13,14
0,0,[ижевск],"[мастер, контрольный, участка, цеха]","[мила, дружка, семь, верст, не, околица, загов...","[более, ради, оскорбления, чем, в, точном, смы...",[salary5],[team4],[managment3],[career3],[workplace4],...,[team4managment3],[team4career3],[team4workplace4],[team4rest4],[managment3career3],[managment3workplace4],[managment3rest4],[career3workplace4],[career3rest4],[workplace4rest4]
1,1,[москва],"[инженер, лаборатории]","[прибавь, как, раз, так, будет, закричал, он, ...","[промелькнуло, в, нем, как, молния, извините, ...",[salary4],[team4],[managment4],[career1],[workplace1],...,[team4managment4],[team4career1],[team4workplace1],[team4rest1],[managment4career1],[managment4workplace1],[managment4rest1],[career1workplace1],[career1rest1],[workplace1rest1]
2,2,[екатеринбург],"[техник, по, документации, плавучей, атомной, ...","[и, пропустил, ее, вперед, седой, тихон, в, па...","[да, письмо, а, ты, не, знал, гм]",[salary5],[team5],[managment5],[career5],[workplace5],...,[team5managment5],[team5career5],[team5workplace5],[team5rest5],[managment5career5],[managment5workplace5],[managment5rest5],[career5workplace5],[career5rest5],[workplace5rest5]
3,5,"[санкт, петербург]","[техник, полиграфист]","[нет, его, так, не, уломаешь, ни, за, что, гов...","[стоило, ль, например, стараться, интриговать,...",[salary5],[team5],[managment5],[career5],[workplace5],...,[team5managment5],[team5career5],[team5workplace5],[team5rest5],[managment5career5],[managment5workplace5],[managment5rest5],[career5workplace5],[career5rest5],[workplace5rest5]
4,7,[москва],"[инструктор, переводчик, по, обслуживанию, глу...","[нахмурившись, не, столько, на, капитана, скол...","[модной, ходячей, идее, чтобы, тотчас, же, опо...",[salary4],[team5],[managment4],[career2],[workplace5],...,[team5managment4],[team5career2],[team5workplace5],[team5rest2],[managment4career2],[managment4workplace5],[managment4rest2],[career2workplace5],[career2rest2],[workplace5rest2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50871,192815,[челябинск],"[инженер, по, эксплуатации, и, наладке, оборуд...","[нами, повелителем, не, говоря, уже, о, моих, ...","[извините, меня, за, мою, назойливость, беспок...",[salary2],[team2],[managment1],[career1],[workplace1],...,[team2managment1],[team2career1],[team2workplace1],[team2rest1],[managment1career1],[managment1workplace1],[managment1rest1],[career1workplace1],[career1rest1],[workplace1rest1]
50872,192818,"[санкт, петербург]","[инженер, физик, в, ракетно, космической, пром...","[задумался, и, поморщился, анна, михайловна, п...","[нему, посмотреть, только, ничего, вздор, все,...",[salary5],[team3],[managment4],[career3],[workplace5],...,[team3managment4],[team3career3],[team3workplace5],[team3rest2],[managment4career3],[managment4workplace5],[managment4rest2],[career3workplace5],[career3rest2],[workplace5rest2]
50873,192820,"[нижний, новгород]","[лаборант, исследователь, в, области, физики]","[я, вам, прямо, скажу, сказал, князь, василий,...","[все, это, чрезвычайно, интересует, и, почту, ...",[salary2],[team1],[managment3],[career2],[workplace2],...,[team1managment3],[team1career2],[team1workplace2],[team1rest1],[managment3career2],[managment3workplace2],[managment3rest1],[career2workplace2],[career2rest1],[workplace2rest1]
50874,192827,[москва],"[инженер, проектировщик, фотошаблонов, для, пр...","[так, я, не, знаю, неловко, стыдно, а, я, знаю...","[милый, друг, франц, природа, и]",[salary4],[team5],[managment1],[career1],[workplace1],...,[team5managment1],[team5career1],[team5workplace1],[team5rest1],[managment1career1],[managment1workplace1],[managment1rest1],[career1workplace1],[career1rest1],[workplace1rest1]


In [33]:
file_df.keys()

Index(['review_id', 'city', 'position', 'positive', 'negative',
       'salary_rating', 'team_rating', 'managment_rating', 'career_rating',
       'workplace_rating', 'rest_recovery_rating', 'target', 'positive_1',
       'negative_1', 'target_len', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14'],
      dtype='object')

In [34]:
# Все поля соединяем в один текст
k = ['city', 'position', 'salary_rating', 'team_rating', 'managment_rating', 'career_rating',
       'workplace_rating', 'rest_recovery_rating', 'positive_1',
       'negative_1', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14']


corpus = [[] for x in range(file_df.shape[0])]
for ind in k:
    for i, x in enumerate(file_df[ind]):
        corpus[i] = corpus[i] + file_df[ind].iloc[i]
corpus = list(corpus)
corpus_1 = []
for text in corpus:
    string = []
    text = list(text)
    for word in text:
        if (type(word) == type('text') and word != ''):
            string.append(word)
    corpus_1.append(' '.join(string))

In [35]:
file_df['text'] = corpus_1

In [36]:
file_df['text'].isna().sum()

0

In [37]:
file_df.keys()

Index(['review_id', 'city', 'position', 'positive', 'negative',
       'salary_rating', 'team_rating', 'managment_rating', 'career_rating',
       'workplace_rating', 'rest_recovery_rating', 'target', 'positive_1',
       'negative_1', 'target_len', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', 'text'],
      dtype='object')

In [38]:
# удаляем ненужные столбцы из файла для тренировки модели
file_df.drop(columns=['review_id', 'city', 'position', 'positive', 'negative',
       'salary_rating', 'team_rating', 'managment_rating', 'career_rating',
       'workplace_rating', 'rest_recovery_rating', 'positive_1',
       'negative_1', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14'], inplace=True)
file_df

,target,target_len,text
0,0,1,ижевск мастер контрольный участка цеха salary5...
1,8,1,москва инженер лаборатории salary4 team4 manag...
2,0,1,екатеринбург техник по документации плавучей а...
3,0,1,санкт петербург техник полиграфист salary5 tea...
4,8,1,москва инструктор переводчик по обслуживанию г...
...,...,...,...
50871,"3,8",3,челябинск инженер по эксплуатации и наладке об...
50872,8,1,санкт петербург инженер физик в ракетно космич...
50873,8,1,нижний новгород лаборант исследователь в облас...
50874,1,1,москва инженер проектировщик фотошаблонов для ...


In [39]:
file_df['text'][0]

'ижевск мастер контрольный участка цеха salary5 team4 managment3 career3 workplace4 rest4 милыйpos дружкаpos верстаpos околицаpos заговоритьpos князьpos быстроpos самоуверенноpos фамильярноpos проситьpos жаловатьpos князьpos оглядетьpos более радиneg оскорблениеneg чем в точныйneg смыслneg ибоneg такойneg уж характерneg и как расплакатьсяneg дитяneg хоть бы и с голодneg тотчасneg же их битьneg начинатьneg и видетьneg я эдакneg часneg в шестойneg salary5team4 salary5managment3 salary5career3 salary5workplace4 salary5rest4 team4managment3 team4career3 team4workplace4 team4rest4 managment3career3 managment3workplace4 managment3rest4 career3workplace4 career3rest4 workplace4rest4'

In [40]:
# записываем файл, готовый для тренировки модели
path_to_save = r'C:\Users\Dima\Desktop\СберУниверситет\Дипломный проект\Диплом файлы\file_df_set_train_MLP_new.csv'
file_df.to_csv(path_to_save)